In [ ]:
import pandas as pd
from ipywidgets import  (Button, interact, Dropdown, FloatSlider, HBox, HTML, IntSlider, Output, Textarea)
from IPython.display import clear_output, display, Image
import matplotlib 
%matplotlib widget 
import mpld3
mpld3.enable_notebook()
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import os, os.path
import datetime
from datetime import date as dt
import pathlib
from pathlib import Path
from collections import defaultdict
import functools
from os import getenv
from io import StringIO
from io import BytesIO
import boto3
boto_kwargs = {"aws_access_key_id": getenv("AWS_ACCESS_KEY_ID"),
            "aws_secret_access_key":getenv("AWS_SECRET_ACCESS_KEY"),
            "region_name": getenv("AWS_REGION")}
s3_client = boto3.Session(**boto_kwargs).client("s3")

In [ ]:
def get_image_list(site):
    today = dt.today().strftime("%Y-%m-%d")
    days = 15

    this_week = []
    subfolders = []
    for i in range(days):
        that_date = (dt.today() - datetime.timedelta(days=i)).strftime("%Y-%m-%d")
        this_week.append(that_date)
    imgs = []
    for key in s3_client.list_objects(Bucket="1-quality-assurance-test")['Contents']:
        if key['Key'].split('/')[0] in this_week:
            if key['Key'].split('/')[1].startswith('site_IND'):
                if key['Key'].split('/')[2].endswith(".png"):
                    imgs.append(key['Key'])
    return imgs

In [ ]:
def get_pond_ids_list(img):
    for key in s3_client.list_objects(Bucket="1-quality-assurance-test")['Contents']:
        if key['Key'].split('/')[0].startswith(img.split('/')[0]):
            if key['Key'].split('/')[1].startswith(img.split('/')[1]):
                if key['Key'].split('/')[2].endswith(".csv"):
                    csv_obj = s3_client.get_object(Bucket="1-quality-assurance-test", Key=key['Key'])
                    body = csv_obj['Body']
                    csv_string = body.read().decode('utf-8')
                    ponds = pd.read_csv(StringIO(csv_string), index_col=0)
                    list_of_ponds_ids = list(ponds.columns)
    return list_of_ponds_ids

In [ ]:
def get_export_path(img):
    for key in s3_client.list_objects(Bucket="1-quality-assurance-test")['Contents']:
        if key['Key'].split('/')[0].startswith(img.split('/')[0]):
            if key['Key'].split('/')[1].startswith(img.split('/')[1]):
                if key['Key'].split('/')[2].endswith(".csv"):
                    name = key['Key'].split('/')[2]
    return name

In [ ]:
def get_groundtruth_csv_fill_default(img):
    for key in s3_client.list_objects(Bucket="1-quality-assurance-test")['Contents']:
        if key['Key'].split('/')[0].startswith(img.split('/')[0]):
            if key['Key'].split('/')[1].startswith(img.split('/')[1]):
                if key['Key'].split('/')[2].endswith(".csv"):
                    csv_obj = s3_client.get_object(Bucket="1-quality-assurance-test", Key=key['Key'])
                    body = csv_obj['Body']
                    csv_string = body.read().decode('utf-8')
                    ponds = pd.read_csv(StringIO(csv_string), index_col=0)
                    list_of_ponds_ids = list(ponds.columns)
    d = defaultdict(dict)
    d[img]['Pond Id'] =  list_of_ponds_ids
    d[img]['Status'] = ['full']*len(list_of_ponds_ids)
    d[img]['Confidence'] = ['High']*len(list_of_ponds_ids)
    d[img]['Reason'] = ['no data']*len(list_of_ponds_ids)
    date = (Path(img).stem).split('_')[1]
    df = pd.DataFrame({'Status': d[img]['Status'],
                   'Confidence': d[img]['Confidence'],
                   'Reason': d[img]['Reason']},
                  index=d[img]['Pond Id']).T
    df_gt = ponds.append(df)
    df_gt = df_gt[~df_gt.index.duplicated(keep='last')]
    return df_gt

In [ ]:
def multi_label_annotate(examples, options=None):
    
    annotation_dict = defaultdict(dict)
    df_gt = pd.DataFrame()
    
    examples = list(examples)
    all_buttons = []

    current_index = -1
    pond_index = -1

    def set_label_text():
        nonlocal count_label
        if current_index < len(examples):
            count_label.value = (
                '{} images annotated, {} images left<br>'.format(
                    len(annotation_dict), len(examples) - current_index
                )
            )
        else:
            count_label.value = 'Annoation done!<hr>'
            
    def set_pond_text():
        nonlocal count_label_pond, pond_index, current_index
        if current_index < len(examples):
            if pond_index < len(get_pond_ids_list(examples[current_index])):
                count_label_pond.value = (
                    '{} ponds left<br>Pond_id: <code>{}</code><hr>'.format(
                       len(get_pond_ids_list(examples[current_index])) - pond_index, get_pond_ids_list(examples[current_index])[pond_index]
                    )
                )
            else:
                count_label_pond.value = '{} ponds left<br>Move to next image.<hr>'.format(
                    len(get_pond_ids_list(examples[current_index])) - pond_index
                )
        else:
            count_label_pond.value = 'Annoation done!<hr>'
        
    def display_fn(key):
        plt.xticks([]),plt.yticks([])
        plt.rcParams["figure.figsize"] = (10,10)
        with BytesIO() as f:
            s3_client.download_fileobj(Bucket="1-quality-assurance-test", Key=key, Fileobj=f)
            f.seek(0)
            img = plt.imread(f, format='png')
            plt.imshow(img)
        return plt.show()

    def show_next():
        clear_colors()
        nonlocal current_index, pond_index, df_gt
        annotation_dict[examples[current_index]] = (df_gt.reset_index()).values.tolist()
        current_index += 1
        pond_index = 0
        set_label_text()
        set_pond_text()
        if current_index >= len(examples):
            for btn in buttons:
                btn.disabled = True
            with out:
                clear_output()
            return
        with out:
            clear_output(wait=True)
            display_fn(examples[current_index])
            df_gt = get_groundtruth_csv_fill_default(examples[current_index])
    def show_next_pond():
        clear_colors()
        nonlocal current_index, pond_index
        current_index == current_index
        pond_index += 1
        set_label_text()
        set_pond_text()
        if pond_index >= len(get_pond_ids_list(examples[current_index])):
            for btn in buttons:
                btn.disabled = True
            with out:
                clear_output()
                show_next()
            return
        with out:
            clear_output(wait=True)
            display_fn(examples[current_index])
            set_pond_text()
            
    def go_back():
        clear_colors()
        nonlocal current_index
        current_index -= 1
        if current_index < 0:
            print('cannot go back')
            return
        with out:
            clear_output(wait=True)
            try:
                del annotation_dict[examples[current_index]]
                set_label_text()
                display_fn(examples[current_index])
            except KeyError:
                print("Key 'testing' not found")
            
    def add_annotation2(df, index, annotation, task_name, pond):
        df.at[task_name, get_pond_ids_list(examples[index])[pond]] = annotation
            
    def skip(btn):
        show_next()

    def done(btn):
        show_next_pond()

    def back(btn):
        go_back()

    def clear_colors():
        nonlocal all_buttons
        for button in all_buttons:
            button.style.button_color = None

    count_label = HTML()
    set_label_text()
    display(count_label)
    count_label_pond = HTML()
    set_pond_text()
    display(count_label_pond)
    
    
    if type(options) == dict:
        task_type = 'classification'
    else:
        raise ValueError('Invalid options. Must be classification dictionary.')

    if task_type == 'classification':
        for key, value in options.items():
            buttons = []
            print(key)
            for label in value:
                btn = MultiLabelButton(description=label, task_name=key)

                def on_click(label, task_name, btn):
                    # if button has a color, clear it! if not, give it a color!
                    if btn.style.button_color is None:
                        btn.style.button_color = 'lightgreen'
                    else:
                        btn.style.button_color = None
                    add_annotation2(df_gt, current_index, label, task_name, pond_index)

                btn.button.on_click(functools.partial(on_click, label, btn.task_name))
                buttons.append(btn.button)
                all_buttons.append(btn.button)

            box = HBox(buttons)
            display(box)

    print('')
    buttons = []
    
    

    btn = Button(description='next pond')
    btn.on_click(done)
    buttons.append(btn)

    btn = Button(description='back')
    btn.on_click(back)
    buttons.append(btn)

    btn = Button(description='next image')
    btn.on_click(skip)
    buttons.append(btn)

    all_buttons += buttons

    box = HBox(buttons)
    display(box)

    out = Output()
    display(out)

    show_next()
    

    return annotation_dict


class MultiLabelButton(object):
    def __init__(self, description, task_name):
        self.description = description
        self.task_name = task_name
        self.button = Button(description=description)

In [ ]:
def to_s3(bucket,filename, content):
    s3_client.put_object(Bucket=bucket, Key=filename, Body=content)

In [ ]:
def on_button_clicked(b):
    for item in b.items():
        df = item[1]
        date = (Path(item[0]).stem).split('_')[1]
        path = pathlib.PurePath(item[0])
        site = path.parent.name
        df_final = pd.DataFrame(df).set_index(0).rename_axis(None)
        df_final=df_final.drop(df_final.index[2]).rename(index={'Status': date, 'Confidence': 'confidence_'+date, 'Reason': 'reason_'+date})

        s3_client = boto3.Session(**boto_kwargs).client("s3")
        csv_buffer = StringIO()
        df_final.to_csv(csv_buffer)
        dest = "1-quality-assurance-ready/" + date + "/" +site +"/" + get_export_path(item[0])
        to_s3("1-quality-assurance-test",dest, csv_buffer.getvalue())
        print(get_export_path(item[0])+' exported')
    print('All files exported!')

In [ ]:
def annotate(user_id):
    if user_id == 'Zack':
        site = 'site_IND'
        imgs = get_image_list(site)
        annotations = multi_label_annotate(imgs,
            options={'Status':['full','empty', 'unknown'], 'Confidence':['Low','High'],
                    'Reason': ['cloud', 'shadow', 'no data']})
        button = Button(description="Export CSVs")
        display(button)
        def click(a):
            on_button_clicked(annotations)
        button.on_click(click)
    elif user_id == 'Houssam':
        site = 'site_ECU'
        imgs = get_image_list(site)
        annotations = multi_label_annotate(imgs,
            options={'Status':['full','empty', 'unknown'], 'Confidence':['Low','High'],
                    'Reason': ['cloud', 'shadow', 'no data']})
        button = Button(description="Export CSVs")
        display(button)
        def click(a):
            on_button_clicked(annotations)
        button.on_click(click)
    else: 
        print('Please enter your UserID')

In [ ]:
user_ids = ['Houssam', 'Zack']
interact(annotate, user_id=Textarea())